# Problem
In this report we will to create a MultiLayer Perceptron and train it using our train data. After the training part we have to predict our test data result and try to maximize our accuracy.

# MultiLayer Perceptron

***model***

In this code our model can support any number of layers and any number of neurons in the model. The calculations will just continue until no hidden layer is left. 


***initial***

To initialize our model we have to create a weight matrix. The index of this matrix will be generated randomly but in a certain range.


***train***

To train our model we will repeat the training until the termination condition is met (which in this case is the number of iterations). For training our data we will perform forward propagation and backpropagation on all of our training samples to calculate the error and use that to change the weight matrix. This change will be calculated using the learning rate at the beginning of the algorithm.


***predict***


After training our model we will give it our test data and see what it will predict for each input and then compare it with the actual output we got to get accuracy.

In [1]:
import numpy as np
import pandas as pd 
import random 
import math
from numpy import  dot,exp

In [2]:
class MLP:
  
  def __init__(self, layer_list, epoch, learning_rate):
    self.model = layer_list
    self.num_layer = len(layer_list)
    self.epoch = epoch
    self.learning_rate = learning_rate
    self.X_train, self.Y_train = self.read_file("train")
    self.X_test, self.Y_test = self.read_file("test")
    self.weight = [] 

  def read_file(self, file_name):
    f = open(file_name, "r")
    txt  = f.readlines()
    X = []
    Y  = []
    for i in range(len(txt)):
      params = [float(j) for j in txt[i].split()]
      X.append([params[0], params[1]])
      Y.append(params[2])
    return X, Y

  def activation(self, x):
    return (1.0 - np.exp(-2*x))/(1.0 + np.exp(-2*x))

  def activation_der(self, x):
    return (1 + self.activation(x))*(1 - self.activation(x))

  def calculate_weight(self):
    np.random.seed(0)
    for i in range(self.num_layer-1):
      self.weight.append(
       2*np.random.rand(self.model[i] + 1, self.model[i+1]) -1   
      )

  def MLPerceptron(self, x, expected_y):
    result = x
    for i in range(len(self.weight)-1):
      dot_ans = np.dot(result[i], self.weight[i])
      result.append(
          np.concatenate((np.ones(1), np.array(self.activation(dot_ans))))
      )
    last_layer = self.activation(np.dot(result[-1], self.weight[-1]))
    result.append(last_layer)

    error = expected_y - result[-1]
    weight_change = [error * self.activation_der(result[-1])]

    for i in range(self.num_layer-2, 0 , -1):
      error = weight_change[-1].dot(self.weight[i][1:].T)
      error = error*self.activation_der(result[i][1:])
      weight_change.append(error)

    weight_change.reverse()

    for i in range(len(self.weight)):
      layer = result[i].reshape(1, self.model[i]+1)
      self.weight[i] = self.weight[i] + self.learning_rate*layer.T.dot(weight_change[i].reshape(1, self.model[i+1]))
  
  def train_model(self):
    bias = np.ones((1, self.X_train.shape[0]))
    input = np.concatenate((bias.T, self.X_train), axis=1)

    for i in range(self.epoch):
      for i in range(len(self.X_train)):
        self.MLPerceptron(
            x = [input[i]],
            expected_y = self.Y_train[i]
        )
  
  def predict(self, x):
    x_w_bias = np.concatenate((np.ones(1).T, np.array(x)))
    for i in range(len(self.weight)):
      res = self.activation(np.dot(x_w_bias, self.weight[i]))
      x_w_bias = np.concatenate((np.ones(1).T, np.array(res)))
    return x_w_bias[1]

  def test_model(self):
    predicted_result = []
    for test in self.X_test:
        p = self.predict(test)
        if p >0.5:
            predicted_result.append(1)
        else:
            predicted_result.append(0)
    accuracy = 0
    for i in range(len(predicted_result)):
      if predicted_result[i] == self.Y_test[i]:
        accuracy += 1
    print(accuracy/len(predicted_result))

  def start_MLP(self):
    self.calculate_weight()
    self.X_train = np.array(self.X_train)
    self.Y_train = np.array(self.Y_train)
    self.train_model()
    self.test_model()


# Parameters

***learning rate***

Learning rate will determine how fast our parameters will change as in how fast we will move towards our prediction. I found out that if we raise the amount of learning rate it will cause our accuracy to be much less. So I tried lowering it but not too much.

***Layers***

I found out that increasing the number of hidden layers will result in a better outcome but after 2 layers there wasn't much change. And increasing the number of layers didn't cause that much of a difference, only decreased the speed.

# Test Cases
Finally we will run or mlp with the given train and test data and also print the test cases that our network predicted wrong.

In [ ]:
instance = MLP([2,2,2,1], 1000, 0.001)
instance.start_MLP()
print(instance.wrong_results)

0.938
[[-0.633687809518789, 0.469846539860649], [0.020529179999515, -0.741064312936821], [0.230474265066374, 1.32188830412696], [0.208502349528497, -1.50069896375079], [0.504925712957714, 1.33289141417598], [-0.025901124087941, -0.637463169792611], [-0.182554291108849, -0.591716871650174], [-0.299854678639638, 0.081549971845352], [-0.748389402883773, -0.519726810513816], [-0.180832995475311, -1.37166912426394], [1.00480338161153, -2.24461117748409], [-0.287569052828006, 0.104110966761412], [0.22535723175402, -1.73359793273308], [-0.842592943376905, -1.40771658085872], [-0.267573699467927, -1.60862593686852], [0.523300924134902, -1.22786948924675], [0.403799434702408, -1.22460591754875], [0.028905774508546, -0.723527073613366], [0.959271009910824, -2.68560885690102], [0.087625271852143, -1.15046630113484], [0.339420145067856, -1.93150784958821], [0.244887479201154, -1.5344014704552], [0.627082730555767, -1.52627999732123], [-1.38032855874806, -0.501020857904726], [0.314398238897843, -1.